# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 19 2023 1:31PM,264984,16,SSF_C1W25,Sartorius Stedim Filters Inc.,Released
1,Jun 19 2023 1:27PM,264983,10,HH-42696,Hush Hush,Released
2,Jun 19 2023 1:27PM,264983,10,HH-42697,Hush Hush,Released
3,Jun 19 2023 1:27PM,264983,10,HH-42698,Hush Hush,Released
4,Jun 19 2023 1:27PM,264983,10,HH-42699,Hush Hush,Released
5,Jun 19 2023 1:26PM,264982,10,38572,"Digital Brands, LLC",Released
6,Jun 19 2023 1:26PM,264982,10,38573,"Digital Brands, LLC",Released
7,Jun 19 2023 1:26PM,264982,10,38574,"Digital Brands, LLC",Released
8,Jun 19 2023 1:26PM,264981,10,ALK39455,Alikai Health,Released
9,Jun 19 2023 1:26PM,264981,10,ALK39505,Alikai Health,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,264980,Released,1
15,264981,Released,6
16,264982,Released,3
17,264983,Released,4
18,264984,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
264980,NaN,NaN,1.0
264981,NaN,NaN,6.0
264982,NaN,NaN,3.0
264983,NaN,NaN,4.0
264984,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264927,0.0,1.0,0.0
264934,0.0,14.0,1.0
264948,1.0,0.0,2.0
264953,0.0,0.0,26.0
264957,0.0,3.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264927,0,1,0
264934,0,14,1
264948,1,0,2
264953,0,0,26
264957,0,3,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264927,0,1,0
1,264934,0,14,1
2,264948,1,0,2
3,264953,0,0,26
4,264957,0,3,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264927,,1,
1,264934,,14,1
2,264948,1,,2
3,264953,,,26
4,264957,,3,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 19 2023 1:31PM,264984,16,Sartorius Stedim Filters Inc.
1,Jun 19 2023 1:27PM,264983,10,Hush Hush
5,Jun 19 2023 1:26PM,264982,10,"Digital Brands, LLC"
8,Jun 19 2023 1:26PM,264981,10,Alikai Health
14,Jun 19 2023 1:19PM,264980,10,"Zelenko Labs, LLC"
15,Jun 19 2023 1:13PM,264979,10,Methapharm-G
16,Jun 19 2023 1:13PM,264979,10,"Methapharm, Inc."
19,Jun 19 2023 1:12PM,264978,10,"Methapharm, Inc."
21,Jun 19 2023 12:24PM,264976,18,Seqirus
22,Jun 19 2023 12:08PM,264975,20,Else Nutrition


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 19 2023 1:31PM,264984,16,Sartorius Stedim Filters Inc.,,,1
1,Jun 19 2023 1:27PM,264983,10,Hush Hush,,,4
2,Jun 19 2023 1:26PM,264982,10,"Digital Brands, LLC",,,3
3,Jun 19 2023 1:26PM,264981,10,Alikai Health,,,6
4,Jun 19 2023 1:19PM,264980,10,"Zelenko Labs, LLC",,,1
5,Jun 19 2023 1:13PM,264979,10,Methapharm-G,,,4
6,Jun 19 2023 1:13PM,264979,10,"Methapharm, Inc.",,,4
7,Jun 19 2023 1:12PM,264978,10,"Methapharm, Inc.",,,2
8,Jun 19 2023 12:24PM,264976,18,Seqirus,,,1
9,Jun 19 2023 12:08PM,264975,20,Else Nutrition,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 19 2023 1:31PM,264984,16,Sartorius Stedim Filters Inc.,1,,
1,Jun 19 2023 1:27PM,264983,10,Hush Hush,4,,
2,Jun 19 2023 1:26PM,264982,10,"Digital Brands, LLC",3,,
3,Jun 19 2023 1:26PM,264981,10,Alikai Health,6,,
4,Jun 19 2023 1:19PM,264980,10,"Zelenko Labs, LLC",1,,
5,Jun 19 2023 1:13PM,264979,10,Methapharm-G,4,,
6,Jun 19 2023 1:13PM,264979,10,"Methapharm, Inc.",4,,
7,Jun 19 2023 1:12PM,264978,10,"Methapharm, Inc.",2,,
8,Jun 19 2023 12:24PM,264976,18,Seqirus,1,,
9,Jun 19 2023 12:08PM,264975,20,Else Nutrition,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 19 2023 1:31PM,264984,16,Sartorius Stedim Filters Inc.,1,,
1,Jun 19 2023 1:27PM,264983,10,Hush Hush,4,,
2,Jun 19 2023 1:26PM,264982,10,"Digital Brands, LLC",3,,
3,Jun 19 2023 1:26PM,264981,10,Alikai Health,6,,
4,Jun 19 2023 1:19PM,264980,10,"Zelenko Labs, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 19 2023 1:31PM,264984,16,Sartorius Stedim Filters Inc.,1.0,NaN,NaN
1,Jun 19 2023 1:27PM,264983,10,Hush Hush,4.0,NaN,NaN
2,Jun 19 2023 1:26PM,264982,10,"Digital Brands, LLC",3.0,NaN,NaN
3,Jun 19 2023 1:26PM,264981,10,Alikai Health,6.0,NaN,NaN
4,Jun 19 2023 1:19PM,264980,10,"Zelenko Labs, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 19 2023 1:31PM,264984,16,Sartorius Stedim Filters Inc.,1.0,0.0,0.0
1,Jun 19 2023 1:27PM,264983,10,Hush Hush,4.0,0.0,0.0
2,Jun 19 2023 1:26PM,264982,10,"Digital Brands, LLC",3.0,0.0,0.0
3,Jun 19 2023 1:26PM,264981,10,Alikai Health,6.0,0.0,0.0
4,Jun 19 2023 1:19PM,264980,10,"Zelenko Labs, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1854862,24.0,0.0,0.0
15,794858,33.0,3.0,1.0
16,264984,1.0,0.0,0.0
18,264976,1.0,0.0,0.0
19,264927,0.0,1.0,0.0
20,1059827,8.0,14.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1854862,24.0,0.0,0.0
1,15,794858,33.0,3.0,1.0
2,16,264984,1.0,0.0,0.0
3,18,264976,1.0,0.0,0.0
4,19,264927,0.0,1.0,0.0
5,20,1059827,8.0,14.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,24.0,0.0,0.0
1,15,33.0,3.0,1.0
2,16,1.0,0.0,0.0
3,18,1.0,0.0,0.0
4,19,0.0,1.0,0.0
5,20,8.0,14.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,24.0
1,15,Released,33.0
2,16,Released,1.0
3,18,Released,1.0
4,19,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20
Status,,,,,,
Completed,0.0,1.0,0.0,0.0,0.0,0.0
Executing,0.0,3.0,0.0,0.0,1.0,14.0
Released,24.0,33.0,1.0,1.0,0.0,8.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20
0,Completed,0.0,1.0,0.0,0.0,0.0,0.0
1,Executing,0.0,3.0,0.0,0.0,1.0,14.0
2,Released,24.0,33.0,1.0,1.0,0.0,8.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20
0,Completed,0.0,1.0,0.0,0.0,0.0,0.0
1,Executing,0.0,3.0,0.0,0.0,1.0,14.0
2,Released,24.0,33.0,1.0,1.0,0.0,8.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()